In [4]:
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from api import api_key

In [15]:
#Real Time Data
ts = TimeSeries(key=api_key, output_format='pandas')
data, meta_data =ts.get_intraday(symbol=tickers, interval='60min', outputsize='2yr')


In [17]:
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2018-10-04 14:30:00,227.4900,228.5000,227.3800,227.8530,3090929.0
2018-10-04 15:30:00,227.8300,228.6200,227.4000,227.8700,2853870.0
2018-10-05 09:30:00,227.9600,228.1325,225.5773,226.4000,6235533.0
2018-10-05 10:30:00,226.4200,226.7400,223.8800,224.4731,4534853.0
2018-10-05 11:30:00,224.4456,224.8900,223.5500,224.3500,3483228.0
2018-10-05 12:30:00,224.3450,224.3450,220.5800,221.5200,5774042.0
2018-10-05 13:30:00,221.5500,224.3200,221.4550,224.2400,4319420.0
2018-10-05 14:30:00,224.2800,225.3400,223.8800,224.6400,3778393.0
2018-10-05 15:30:00,224.6425,225.0000,223.8300,224.1400,3256588.0


In [6]:
#Daily Data
tickers = ['AAPL']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2015-01-01'
end_date = '2018-10-24'

dict = {}
# User pandas_reader.data.DataReader to load the desired data. As simple as that.

for ticker in tickers:
    panel_data = data.DataReader(ticker, 'iex', start_date, end_date)
    panel_data["Ticker"] = ticker
    panel_data["Moving Average"] = panel_data["close"].rolling(window=5).mean()

In [19]:
panel_data

,open,high,low,close,volume,Ticker,Moving Average
date,,,,,,,
2015-01-02,104.2471,104.2939,100.4662,102.3192,53204626,AAPL,NaN
2015-01-05,101.3459,101.6828,98.6506,99.4367,64285491,AAPL,NaN
2015-01-06,99.7082,100.5411,97.9206,99.4461,65797116,AAPL,NaN
2015-01-07,100.3258,101.2617,99.8532,100.8406,40105934,AAPL,NaN
2015-01-08,102.2257,104.9584,101.7296,104.7151,59364547,AAPL,101.35154
2015-01-09,105.4451,105.9879,103.1428,104.8274,53699527,AAPL,101.85318
2015-01-12,105.3796,105.4076,101.8232,102.2444,49650790,AAPL,102.41472
2015-01-13,104.2846,105.5667,101.9262,103.1522,67091928,AAPL,103.15594
2015-01-14,102.0478,103.4049,101.5425,102.7591,48956588,AAPL,103.53964
